In [2]:
import re
import nltk
import random
import numpy as np
import pandas as pd
import langid

from better_profanity import profanity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [3]:
paracrawl = pd.read_csv("/auc/thesis/data/paracrawl.txt", encoding='utf-8', on_bad_lines='skip', sep='\t', names=['English', 'Gaeilge'], dtype=str)
paracrawl

FileNotFoundError: [Errno 2] No such file or directory: '/auc/thesis/data/paracrawl.txt'

In [19]:
def filtering(en, ga):
    en, ga = str(en), str(ga)
    i = len(en)
    j = len(ga)
    
    # length matching as per the Speechmatics Parallel Corpus Filtering System for WMT18
    #f len(en) < 100:
    if 6 * i > j and i < 6 * j:
        if i < 3 or j < 3 or (i < 2.2 * j and j < 2.2 * i):
            if i < 10 or j < 10 or (i < 2 * j and j < 2 * i):
                
                # removing profanity
                if not profanity.contains_profanity(en):
                    
                    # checking bleu similarity
                    smoothie = SmoothingFunction().method7
                    score = sentence_bleu([en], ga, smoothing_function=smoothie)
                    if score < 0.5:

                        # is it Irish?
                        if langid.classify(ga)[0] == 'ga':
                            
                            return True  
    return False        

In [21]:
def sub(text):
    return re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z\u00C0-\u017F \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

zipped = list(zip(paracrawl[:1000]["English"], paracrawl[:1000]["Gaeilge"]))

filtered = [(sub(en), sub(ga)) for (en, ga) in zipped if filtering(en, ga)]

pairs = pd.DataFrame(filtered, columns=["English", "Gaeilge"])
pairs.to_csv("paracrawlv2.csv")
pairs

,English,Gaeilge
0,[GA] (b) is established for the benefit of ind...,[EN] (b) gur bunaíodh an scéim nó an chuid réa...
1,You must complete the application form giving ...,Ní mór duit an fhoirm iarratais a líonadh le s...
2,Media You blocked @ConservativeCh6 Are you sur...,Meáin Chuir tú cosc ar @ConservativeCh6 An bhf...
3,the dog becomes naughty and nervous.,thiocfaidh an madra go dona agus neirbhíseach.
4,(4) Where a member of the Commission becomes a...,(4) Má thagann comhalta den Choimisiún chun bh...
...,...,...
794,The Commission shall adopt measures pursuant t...,Déanfaidh an Coimisiún bearta chur chun feidhm...
795,"When mounted in the window, the perforated gra...","Nuair a bheidh sé suite sa fhuinneog, ní chuir..."
796,We also know Replica IWC Watches that have exc...,Tá a fhios againn freisin Macasamhail IWC Fair...
797,Edouard PhilippeVerified account @EPhilippePM ...,Olivier DussoptCuntas deimhnithe @olivierdusso...


In [38]:
for i, n in enumerate(zipped):
    print(n, "=", filtering(zipped[i][0], zipped[i][1]), "\n")

('End of conversation New conversation William Ruby @Bill_Ruby 23h23 hours ago', 'Deireadh an chomhrá Comhrá nua William Ruby @Bill_Ruby 23 u23 uair an chloig ó shin') = False 

('[GA] (b) is established for the benefit of individuals engaged in or connected with a particular occupation (or one or other of a group of occupations), and for the purpose of providing retirement annuities for them, with or without subsidiary benefits for their families or dependants, and', "[EN] (b) gur bunaíodh an scéim nó an chuid réamhráite den scéim ar mhaithe le pearsana aonair a bhíonn ag gabhail do shlí bheatha áirithe, nó ag a bhfuil baint le slí bheatha áirithe (nó an ceann seo nó an ceann siúd de ghrúpa slite beatha), agus chun blianachtaí scoir a sholáthar dóibh, i dteannta nó d'éagmais foshochar dá dteaghlaigh nó dá gcleithiúnaithe, agus") = True 

('You must complete the application form giving us the details of your event.', 'Ní mór duit an fhoirm iarratais a líonadh le sonraí a thabhairt dúin